In [1]:
import pandas as pd
import json
import re
import unicodedata

df=pd.read_json("faculty_finder/faculty_output.json")
print(f"Data loaded: {len(df)} rows")


Data loaded: 111 rows


In [2]:
df.head(5)

,faculty_type,name,education,phone,address,email,specializations,biography,teaching,research,publications,website_links
0,faculty,Yash vasavada,"PhD (Electrical Engineering), Virginia Polytec...",079-68261634,"# 1224, FB-1, DA-IICT, Gandhinagar, Gujarat, I...",yash_vasavada[at]dau[dot]ac[dot]in,"Communication, Signal Processing, Machine Lear...",Yash Vasavada is currently a Professor at DAII...,"[Introduction to Communication Systems, Advanc...","Communication, Signal Processing, Machine Lear...","[, Michael Parr, Nidhi Sindhav, and Saumi S., ...",[http://intranet.daiict.ac.in/~yash_vasavada/i...
1,faculty,Yash agrawal,"PhD (Electronics & Communication), NIT Hamirpur","079-68261629, 9882114669","# 1101, FB-1, DA-IICT, Gandhinagar, Gujarat, I...","yash_agrawal[at]dau[dot]ac[dot]in, mr[dot]yash...","VLSI, Nanotechnology, Numerical Method Techniq...",Dr. Yash Agrawal received his Ph.D. and M.Tech...,"[Digital Logic Design, Digital Design using HD...","VLSI, Nanotechnology, Numerical Method Techniq...","[Y. Agrawal, M. Girish, and R. Chandel, “An ef...",[http://www.linkedin.com/in/yash-agrawal-525b0...
2,faculty,Vinay palaparthy,"PhD (Electrical Engineering), IIT Bombay",079-68261677,"# 1208, FB-1, DA-IICT, Gandhinagar, Gujarat, I...",vinay_shrinivas[at]dau[dot]ac[dot]in,"Micro-Electro-Mechanical Systems (MEMS), Physi...",Vinay Palaparthy is working as the associate p...,"[Introduction to MEMS, Sensors and Instrumenta...","Micro-Electro-Mechanical Systems (MEMS), Physi...","[G. Bhatti, Y. Agrawal,, R. Sharma and M. G. K...",[]
3,faculty,Tathagata bandyopadhyay,"PhD (Statistics), University of Calcutta, Kolkata",079-68261572,,tathagata_b[at]dau[dot]ac[dot]in,"Statistical Inference, Survey Sampling, Discre...",Dr. Tathagata Bandyopadhyay has been in the ac...,"[Research Methodology, Data Analytics, Bayesia...","Statistical Inference, Survey Sampling, Discre...","[Arindam Chatterjee,, , Ayoushman Bhattacharya...",[]
4,faculty,Tapas kumar maiti,PhD (Electronics & Telecommunication Engineeri...,079-68261637,"# 2206, FB-2, DA-IICT, Gandhinagar, Gujarat, I...",tapas_kumar[at]dau[dot]ac[dot]in,Meet Dr. Tapas Kumar Maiti: Pioneering Researc...,"Dr. Tapas Kumar Maiti, a former associate prof...","[IE416: Robot Programming (Winter 2024), IE410...",Meet Dr. Tapas Kumar Maiti: Pioneering Researc...,"[T. K. Maiti, S. Dutta, Y. Ochi, M. Miura-Matt...","[https://sites.google.com/view/cybernoid, http..."


In [3]:
def title_case_name(name):
    if not isinstance(name, str) or name.strip() == "":
        return None
    return name.title()

df['name'] = df['name'].apply(title_case_name)

In [4]:
def clean_email(email):
    if not isinstance(email, str) or not email.strip():
        return None

    email = email.replace("[at]", "@").replace("[dot]", ".")
    email = email.replace(" ", "")

    emails = [e for e in email.split(",") if "@" in e]

    return emails if emails else None

df['email'] = df['email'].apply(clean_email)

In [5]:

def clean_text(text):
    if not isinstance(text, str) or not text.strip():
        return None

    # Normalizing unicode (fixes most encoding issues)
    text = unicodedata.normalize("NFKC", text)

    # Remove non-printable characters
    text = "".join(ch for ch in text if ch.isprintable())

    # Normalizing whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text

df['education'] = df['education'].apply(clean_text)
df['address'] = df['address'].apply(clean_text)
df['specializations']= df['specializations'].apply(clean_text)
df['biography'] = df['biography'].apply(clean_text)
df['research']=df['research'].apply(clean_text)


In [6]:
def clean_and_categorize_phone(phone):
    if not isinstance(phone, str) or not phone.strip():
        return None
    
    # Remove hyphens and spaces, then extract digit groups
    phone_cleaned = phone.replace("-", "").replace(" ", "")
    result = {"mobile": [], "landline": []}
    
    for num in re.findall(r"\d+", phone_cleaned):
        if len(num) == 10 and num[0] in "6789":
            result["mobile"].append(num)
        elif num.startswith("0") and 10 <= len(num) <= 12:
            result["landline"].append(num)
    
    result = {k: v for k, v in result.items() if v}
    return result if result else None

df['phone'] = df['phone'].apply(clean_and_categorize_phone)

In [7]:
def clean_list(lst):
    if not isinstance(lst, list):
        return None

    cleaned = [
        clean_text(item)
        for item in lst
        if isinstance(item, str) and clean_text(item)
    ]

    return cleaned if cleaned else None

df['teaching'] = df['teaching'].apply(clean_list)

In [8]:
def clean_publications(pub_list):
    """Merging broken publication fragments"""
    if not isinstance(pub_list, list):
        return None

    publications = []
    buffer = ""

    for item in pub_list:
        if not isinstance(item, str):
            continue

        item = clean_text(item)
        if not item:
            continue

        buffer += " " + item

        if re.search(r"\b(19|20)\d{2}\b", buffer) or "doi" in buffer.lower():
            publications.append(buffer.strip())
            buffer = ""

    if buffer.strip():
        publications.append(buffer.strip())

    return publications if publications else None

df["publications"] = df["publications"].apply(clean_publications)

In [9]:
def clean_links(links):
    if not isinstance(links, list):
        return None

    links = [l.strip() for l in links if isinstance(l, str) and l.startswith("http")]
    return links if links else None

df["website_links"] = df["website_links"].apply(clean_links)

In [10]:
def categorize_links(links):
    if not isinstance(links, list) or not links:
        return None

    categorized = {
        "personal_website": [],
        "google_scholar": [],
        "linkedin": [],
        "youtube": [],
        "other": []
    }

    for link in links:
        if "scholar.google" in link:
            categorized["google_scholar"].append(link)

        elif "linkedin.com" in link:
            categorized["linkedin"].append(link)

        elif "youtube.com" in link or "youtu.be" in link:
            categorized["youtube"].append(link)

        elif "sites.google" in link or "github.io" in link:
            categorized["personal_website"].append(link)

        else:
            categorized["other"].append(link)

    categorized = {k: v for k, v in categorized.items() if v}

    return categorized if categorized else None

df["website_links"] = df["website_links"].apply(categorize_links)


In [11]:
# normalizing all values including empty strings to None
df = df.applymap(
    lambda x: None if isinstance(x, str) and not x.strip() else x
)
df = df.where(pd.notnull(df), None)

C:\Users\ambuj\AppData\Local\Temp\ipykernel_22628\3487208760.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(


In [12]:
df.to_csv(
    "faculty_cleaned.csv",
    index=False
)

print("Cleaning completed and saved to faculty_cleaned.csv")

Cleaning completed and saved to faculty_cleaned.csv


In [13]:
df.head(5)

,faculty_type,name,education,phone,address,email,specializations,biography,teaching,research,publications,website_links
0,faculty,Yash Vasavada,"PhD (Electrical Engineering), Virginia Polytec...",{'landline': ['07968261634']},"# 1224, FB-1, DA-IICT, Gandhinagar, Gujarat, I...",[yash_vasavada@dau.ac.in],"Communication, Signal Processing, Machine Lear...",Yash Vasavada is currently a Professor at DAII...,"[Introduction to Communication Systems, Advanc...","Communication, Signal Processing, Machine Lear...","[, Michael Parr, Nidhi Sindhav, and Saumi S., ...",{'google_scholar': ['https://scholar.google.co...
1,faculty,Yash Agrawal,"PhD (Electronics & Communication), NIT Hamirpur","{'mobile': ['9882114669'], 'landline': ['07968...","# 1101, FB-1, DA-IICT, Gandhinagar, Gujarat, I...","[yash_agrawal@dau.ac.in, mr.yashagrawal@gmail....","VLSI, Nanotechnology, Numerical Method Techniq...",Dr. Yash Agrawal received his Ph.D. and M.Tech...,"[Digital Logic Design, Digital Design using HD...","VLSI, Nanotechnology, Numerical Method Techniq...","[Y. Agrawal, M. Girish, and R. Chandel, “An ef...",{'linkedin': ['http://www.linkedin.com/in/yash...
2,faculty,Vinay Palaparthy,"PhD (Electrical Engineering), IIT Bombay",{'landline': ['07968261677']},"# 1208, FB-1, DA-IICT, Gandhinagar, Gujarat, I...",[vinay_shrinivas@dau.ac.in],"Micro-Electro-Mechanical Systems (MEMS), Physi...",Vinay Palaparthy is working as the associate p...,"[Introduction to MEMS, Sensors and Instrumenta...","Micro-Electro-Mechanical Systems (MEMS), Physi...","[G. Bhatti, Y. Agrawal, R. Sharma and M. G. Ku...",None
3,faculty,Tathagata Bandyopadhyay,"PhD (Statistics), University of Calcutta, Kolkata",{'landline': ['07968261572']},None,[tathagata_b@dau.ac.in],"Statistical Inference, Survey Sampling, Discre...",Dr. Tathagata Bandyopadhyay has been in the ac...,"[Research Methodology, Data Analytics, Bayesia...","Statistical Inference, Survey Sampling, Discre...","[Arindam Chatterjee, , Ayoushman Bhattacharya ...",None
4,faculty,Tapas Kumar Maiti,PhD (Electronics & Telecommunication Engineeri...,{'landline': ['07968261637']},"# 2206, FB-2, DA-IICT, Gandhinagar, Gujarat, I...",[tapas_kumar@dau.ac.in],Meet Dr. Tapas Kumar Maiti: Pioneering Researc...,"Dr. Tapas Kumar Maiti, a former associate prof...","[IE416: Robot Programming (Winter 2024), IE410...",Meet Dr. Tapas Kumar Maiti: Pioneering Researc...,"[T. K. Maiti, S. Dutta, Y. Ochi, M. Miura-Matt...",{'personal_website': ['https://sites.google.co...


In [14]:
JSON_COLUMNS = [
    "phone",
    "email",
    "teaching",
    "publications",
    "website_links"
]

def to_json_safe(value):
    if isinstance(value, (list, dict)):
        return json.dumps(value, ensure_ascii=False)
    return None if pd.isna(value) else value

for col in JSON_COLUMNS:
    df[col] = df[col].apply(to_json_safe)

In [15]:
from collections import defaultdict

column_types = defaultdict(set)

for col in df.columns:
    for val in df[col]:
        if val is not None:
            column_types[col].add(type(val).__name__)

column_types


defaultdict(set,
            {'faculty_type': {'str'},
             'name': {'str'},
             'education': {'str'},
             'phone': {'str'},
             'address': {'str'},
             'email': {'str'},
             'specializations': {'str'},
             'biography': {'str'},
             'teaching': {'str'},
             'research': {'str'},
             'publications': {'str'},
             'website_links': {'str'}})

In [16]:
import sqlite3

conn = sqlite3.connect("faculty.db")
cursor = conn.cursor()
try :
    cursor.execute("DROP TABLE IF EXISTS faculty")

    cursor.execute("""
    CREATE TABLE faculty (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        faculty_type TEXT,
        name TEXT,
        education TEXT,
        address TEXT,
        specializations TEXT,
        biography TEXT,
        research TEXT,
        phone TEXT,
        email TEXT,
        teaching TEXT,
        publications TEXT,
        website_links TEXT
    )""")

    conn.commit()
    conn.close()
except Exception as e:
    print(f"Error creating database table: {e}")

In [21]:
# df = df.reset_index(drop=True)
# df.insert(0, "id", range(1, len(df) + 1))

df['faculty_type'].unique()

array(['faculty', 'adjunct-faculty', 'adjunct-faculty-international',
       'distinguished-professor', 'professor-practice'], dtype=object)

In [18]:
try:
    from sqlalchemy import create_engine

    engine = create_engine("sqlite:///faculty.db")

    df.to_sql(
        "faculty",
        con=engine,
        if_exists="replace",
        index=False
    )
    print("Data inserted into faculty.db successfully.")
except Exception as e:
    print(f"Error inserting data into database: {e}")


Data inserted into faculty.db successfully.


In [19]:
try:
    conn = sqlite3.connect("faculty.db")

    result = pd.read_sql(
        "SELECT * FROM faculty LIMIT 3",
        conn
    )

    print(result)
    conn.close()
except Exception as e:
    print(f"Error querying database: {e}")

   id faculty_type              name  \
0   1      faculty     Yash Vasavada   
1   2      faculty      Yash Agrawal   
2   3      faculty  Vinay Palaparthy   

                                           education  \
0  PhD (Electrical Engineering), Virginia Polytec...   
1    PhD (Electronics & Communication), NIT Hamirpur   
2           PhD (Electrical Engineering), IIT Bombay   

                                               phone  \
0                      {"landline": ["07968261634"]}   
1  {"mobile": ["9882114669"], "landline": ["07968...   
2                      {"landline": ["07968261677"]}   

                                             address  \
0  # 1224, FB-1, DA-IICT, Gandhinagar, Gujarat, I...   
1  # 1101, FB-1, DA-IICT, Gandhinagar, Gujarat, I...   
2  # 1208, FB-1, DA-IICT, Gandhinagar, Gujarat, I...   

                                               email  \
0                        ["yash_vasavada@dau.ac.in"]   
1  ["yash_agrawal@dau.ac.in", "mr.yashagrawal@gma...